In [1]:
import pandas as pd
from krisk import init_notebook; init_notebook
import krisk.plot as plt

In [2]:
init_notebook()

<IPython.core.display.Javascript object>

In [3]:
df = pd.read_csv('krisk/tests/data/gapminderDataFiveYear.txt', sep='\t')

In [9]:
p.set_theme('dark')
p.set_size(width=1000,height=700)
p.to_html('../dark_bar.html')

In [ ]:
%load krisk/plot.py

In [24]:
from bokeh.charts import Bar

In [32]:
p.to_html('../this2.html')

In [22]:
p.to_html('')

In [27]:
%%file krisk/make_chart.py

from krisk.chart import Chart
from copy import deepcopy

def round_list(arr):
    try:
        return arr.values.round(3).tolist() # Numeric Array
    except TypeError:
        try:
            return arr.unique().tolist() #String Array
        except AttributeError:
            return (arr.apply(lambda x: x.values.round(3) #Dataframe
                            if x.dtype.name.startswith('float') 
                            else x)
                    .values.tolist())
    
                
def make_chart(df,**kwargs):
    
    def insert_series_on(f):

        if category:
            #Iterate and append Data for every category
            for cat, subset in df.groupby(category):
                cat = str(cat)
                insert_data_to_series(f,subset,cat)
                c._option['legend']['data'].append(cat)
        else:
            insert_data_to_series(f,df)
    
    
    def insert_data_to_series(f,df,cat=None):
        data = f(df)
        series = deepcopy(elem_series)
        series['data'] = round_list(data)
        series['type'] = kwargs['type']
        series['name'] = cat if cat else x
        c._option['series'].append(series)
        
    
    c = Chart(**kwargs)
    
    elem_series = {
            'name': '',
            'type': kwargs['type'],
            'data': []}
    
    x = kwargs['x']
    y = kwargs.get('y')
    category = kwargs['category']
    
    
    def bar_line_hist_condition():
        """Provide stacked,annotate, area for bar line hist"""
        series = c._option['series']

        d_annotate = {'normal':{'show':True,
                                'position':'top'}}

        if category and kwargs['stacked'] == True:
            for s in series:
                s['stack'] = category

                if kwargs['type'] == 'line' and kwargs['area'] == True:
                    s['areaStyle'] = {'normal': {}}

                if kwargs['annotate'] == 'all':

                    if kwargs['type'] == 'bar':
                        d_ant = deepcopy(d_annotate)
                        d_ant['normal']['position'] = 'inside'
                        s['label'] = deepcopy(d_ant)
                    else:
                        s['label'] = deepcopy(d_annotate)
                        
        if kwargs['annotate'] == 'top':
            series[-1]['label'] = d_annotate
        # TODO: make annotate receive all kinds supported in echarts.
    
    if kwargs['type'] in ['bar','line']:
       
        
        
        def get_bar_line_data(df):
            c._option['xAxis']['data'] = round_list(df[x].drop_duplicates())
#             c._option['yAxis']['scale'] = True
            
            if y is None:
                data = df[x].value_counts()
            else:
                data = (df[y]
                        if kwargs['how'] is None else
                        df.groupby(x)[y].aggregate(kwargs['how']))
                    
                
#             data = (df[x].value_counts()
#                     if y is None else
#                     df.groupby(x)[y].aggregate(kwargs['how']))
            
            return data
            
        
        insert_series_on(get_bar_line_data)
        bar_line_hist_condition()
            
    elif kwargs['type'] == 'hist':
        kwargs['type'] = 'bar'
        
        
        def get_hist_data(df):
            
            y_val,x_val = np.histogram(df[x],
                                       bins=kwargs['bins'],
                                       normed=kwargs['normed'])
            data = pd.Series(y_val)
            bins = x_val.astype(int).tolist()
            c._option['xAxis']['data'] = bins
            
            return data
        
        insert_series_on(get_hist_data)
        bar_line_hist_condition()
            
    elif kwargs['type'] == 'scatter':
        
        c._option['xAxis'] = {'type': 'value',
                              'name': x,
                              'max': int(df[x].max())}
        c._option['yAxis'] = {'type': 'value',
                              'name': y,
                              'max': int(df[y].max())}
        c._option['visualMap'] = []
        
        
        
        cols = [x,y]
        size = kwargs['size']
        if size is not None:
            vmap_template_size = {
                                    'show': False,
                                    'dimension': 2,
                                    'min': 0,
                                    'max': 250,
                                    'precision': 0.1,
                                    'inRange': {
                                        'symbolSize': [10, 70]
                                    }
                                }
            vmap_size = deepcopy(vmap_template_size)
            vmap_size['min'] = df[size].min()
            vmap_size['max'] = df[size].max()
#             vmap_size['inRange']['symbolSize'] = [30,100]
            c._option['visualMap'].append(vmap_size)
            cols.append(size)
        
            
        #TODO: Fix Saturate
#          saturate = kwargs['saturate']
#         if saturate is not None:
#             vmap_saturate = deepcopy(visual_map_template)
#             vmap_saturate['min'] = float(df[saturate].min())
#             vmap_saturate['max'] = float(df[saturate].max())
#             vmap_saturate['inRange']['colorLightness'] = [1,0.5]
#             c._option['visualMap'].append(vmap_saturate)
#             cols.append(saturate)

        columns = cols+df.columns.difference(cols).tolist()
        c._kwargs_chart_['columns'] = columns
        
        def get_scatter_data(df):
            data = df[columns]
#             print(columns)
            return data
        
        insert_series_on(get_scatter_data)
    
        
            
    return c

Overwriting krisk/make_chart.py


In [28]:
pd.read_csv()

In [29]:
import seaborn as sns

In [37]:
%%file krisk/plot.py

from krisk.make_chart import make_chart

def bar(df,x,y=None,category=None,how='count',stacked=False,
        annotate=None,**kwargs):
    """
    Parameters
    ----------
    df: pd.DataFrame
        data to be used for the chart
    x: string
        columns to be used as category axis
    y: string, default to None
        if None, use count of category value. otherwise aggregate based on y columns
    category: string, default to None
        another grouping columns inside x-axis
    how: string, default to None
        to be passed to pd.group_by(x).aggregate(how). Can be mean,median, or any 
        reduced operations.
    stacked: Boolean, default to False.
        Whether to stacked category on top of the other categories.
    annotate: string, {'all',True} default to None
        if True, annotate value on top of the plot element. If stacked is also True, annotate the last
        category. if 'all' and stacked, annotate all category 
    
    Returns
    -------
    Chart Object
    """
    
    # TODO: add optional argument trendline
    
    kwargs['x'] = x
    kwargs['y'] = y
    kwargs['category'] = category
    kwargs['how'] = how
    kwargs['type'] = 'bar'
    kwargs['stacked'] = stacked
    kwargs['annotate'] = 'top' if annotate == True else annotate
    
    return make_chart(df,**kwargs)

def line(df,x,y=None,category=None,how=None,stacked=False,area=False,
         annotate=None,**kwargs):
    """
    Parameters
    ----------
    df: pd.DataFrame
        data to be used for the chart
    x: string
        columns to be used as category axis
    y: string, default to None
        if None, use count of category value. otherwise aggregate based on y columns
    category: string, default to None
        another grouping columns inside x-axis
    how: string, default to None
        to be passed to pd.group_by(x).aggregate(how). Can be mean,median, or any 
        reduced operations.
    stacked: Boolean, default to False.
        Whether to stacked category on top of the other categories.
    annotate: string, {'all',True} default to None
        if True, annotate value on top of the plot element. If stacked is also True, annotate the last
        category. if 'all' and stacked, annotate all category 
    
    Returns
    -------
    Chart Object
    """
    
    kwargs['x'] = x
    kwargs['y'] = y
    kwargs['category'] = category
    kwargs['how'] = how
    kwargs['type'] = 'line'
    kwargs['stacked'] = stacked
    kwargs['area'] = area
    kwargs['annotate'] = 'top' if annotate == True else annotate
    
    return make_chart(df,**kwargs)

def hist(df,x,category=None,bins=10,normed=False,stacked=False,
         annotate=None,**kwargs):
    """
    Parameters
    ----------
    df: pd.DataFrame
        data to be used for the chart
    x: string
        columns to be used as category axis
    category: string, default to None
        another grouping columns inside x-axis
    bins: int, default to 10
        Set number of bins in histogram
    normed: boolean, default to False
        Whether normalize the histogram
    stacked: Boolean, default to False.
        Whether to stacked category on top of the other categories.
    annotate: string, {'all',True} default to None
        if True, annotate value on top of the plot element. If stacked is also True, annotate the last
        category. if 'all' and stacked, annotate all category 
    
    Returns
    -------
    Chart Object
    """
    
    kwargs['x'] = x
    kwargs['category'] = category
    kwargs['bins'] = bins
    kwargs['type'] = 'hist'
    kwargs['normed'] = normed
    kwargs['stacked'] = stacked
    kwargs['annotate'] = 'top' if annotate == True else annotate
    
    return make_chart(df,**kwargs)

def scatter(df,x,y,size=None,category=None,size_px=(10,70),**kwargs):
    """
    Parameters
    ----------
    df: pd.DataFrame
        data to be used for the chart
    x,y: string, columns in pd.DataFrame
        Used as coordinate in scatter chart
    size: string, columns in pd.DataFrame default to None
        Used as sizing value of the scatter points
    category: string, default to None
        column used as grouping color category
    size_px: tuple, default to (10,70)
        boundary size, lower and upper limit in pixel for min-max scatter points
        
    Returns
    -------
    Chart Object
    """
    
    kwargs['x'] = x
    kwargs['y'] = y
    kwargs['category'] = category
    kwargs['size'] = size
    #kwargs['saturate'] = saturate #TODO: Fix saturate
    kwargs['type'] = 'scatter'
    
    return make_chart(df,**kwargs)

Overwriting krisk/plot.py


In [36]:
%%file krisk/make_chart.py

from krisk.chart import Chart
from copy import deepcopy

def round_list(arr):
    try:
        return arr.values.round(3).tolist() # Numeric Array
    except TypeError:
        try:
            return arr.unique().tolist() #String Array
        except AttributeError:
            return (arr.apply(lambda x: x.values.round(3) #Dataframe
                            if x.dtype.name.startswith('float') 
                            else x)
                    .values.tolist())
    
                
def make_chart(df,**kwargs):
    
    def insert_series_on(f):

        if category:
            #Iterate and append Data for every category
            for cat, subset in df.groupby(category):
                cat = str(cat)
                insert_data_to_series(f,subset,cat)
                c._option['legend']['data'].append(cat)
        else:
            insert_data_to_series(f,df)
    
    
    def insert_data_to_series(f,df,cat=None):
        data = f(df)
        series = deepcopy(elem_series)
        series['data'] = round_list(data)
        series['type'] = kwargs['type']
        series['name'] = cat if cat else x
        c._option['series'].append(series)
        
    
    c = Chart(**kwargs)
    
    elem_series = {
            'name': '',
            'type': kwargs['type'],
            'data': []}
    
    x = kwargs['x']
    y = kwargs.get('y')
    category = kwargs['category']
    
    
    def bar_line_hist_condition():
        """Provide stacked,annotate, area for bar line hist"""
        series = c._option['series']

        d_annotate = {'normal':{'show':True,
                                'position':'top'}}

        if category and kwargs['stacked'] == True:
            for s in series:
                s['stack'] = category

                if kwargs['type'] == 'line' and kwargs['area'] == True:
                    s['areaStyle'] = {'normal': {}}

                if kwargs['annotate'] == 'all':

                    if kwargs['type'] == 'bar':
                        d_ant = deepcopy(d_annotate)
                        d_ant['normal']['position'] = 'inside'
                        s['label'] = deepcopy(d_ant)
                    else:
                        s['label'] = deepcopy(d_annotate)
                        
        if kwargs['annotate'] == 'top':
            series[-1]['label'] = d_annotate
        # TODO: make annotate receive all kinds supported in echarts.
    
    if kwargs['type'] in ['bar','line']:
       
        
        
        def get_bar_line_data(df):
            c._option['xAxis']['data'] = round_list(df[x].drop_duplicates())
#             c._option['yAxis']['scale'] = True #TODO: Still need to be evaluated
            
            if y is None:
                data = df[x].value_counts()
            else:
                data = (df[y]
                        if kwargs['how'] is None else
                        df.groupby(x)[y].aggregate(kwargs['how']))
                    
            # TODO: Still need to be evaluated
#             data = (df[x].value_counts()
#                     if y is None else
#                     df.groupby(x)[y].aggregate(kwargs['how']))
            
            return data
            
        
        insert_series_on(get_bar_line_data)
        bar_line_hist_condition()
            
    elif kwargs['type'] == 'hist':
        kwargs['type'] = 'bar'
        
        
        def get_hist_data(df):
            
            y_val,x_val = np.histogram(df[x],
                                       bins=kwargs['bins'],
                                       normed=kwargs['normed'])
            data = pd.Series(y_val)
            bins = x_val.astype(int).tolist()
            c._option['xAxis']['data'] = bins
            
            return data
        
        insert_series_on(get_hist_data)
        bar_line_hist_condition()
            
    elif kwargs['type'] == 'scatter':
        
        c._option['xAxis'] = {'type': 'value',
                              'name': x,
                              'max': int(df[x].max())}
        c._option['yAxis'] = {'type': 'value',
                              'name': y,
                              'max': int(df[y].max())}
        c._option['visualMap'] = []
        
        
        
        cols = [x,y]
        size = kwargs['size']
        if size is not None:
            vmap_template_size = {
                                    'show': False,
                                    'dimension': 2,
                                    'min': 0,
                                    'max': 250,
                                    'precision': 0.1,
                                    'inRange': {
                                        'symbolSize': [10, 70]
                                    }
                                }
            vmap_size = deepcopy(vmap_template_size)
            vmap_size['min'] = df[size].min()
            vmap_size['max'] = df[size].max()  
            vmap_size['inRange']['symbolSize'] = list(kwargs['size_px'][:2])
            c._option['visualMap'].append(vmap_size)
            cols.append(size)
        
            
        #TODO: Fix Saturate
#          saturate = kwargs['saturate']
#         if saturate is not None:
#             vmap_saturate = deepcopy(visual_map_template)
#             vmap_saturate['min'] = float(df[saturate].min())
#             vmap_saturate['max'] = float(df[saturate].max())
#             vmap_saturate['inRange']['colorLightness'] = [1,0.5]
#             c._option['visualMap'].append(vmap_saturate)
#             cols.append(saturate)

        columns = cols+df.columns.difference(cols).tolist()
        c._kwargs_chart_['columns'] = columns
        
        def get_scatter_data(df):
            data = df[columns]
#             print(columns)
            return data
        
        insert_series_on(get_scatter_data)
    
        
            
    return c

Overwriting krisk/make_chart.py


In [35]:
(10,20,30)[:2]

(10, 20)

In [235]:
# %%file pandas-echarts/tests/test_option.py


df = pd.read_csv('gapminderDataFiveYear.txt',sep='\t')

def test_bar():
    
    true_option = json.load(open('pandas-echarts/tests/data/bar_option.json','r'))
    p = bar(df,'lifeExp',y='pop',category='continent',how='mean',stacked=True)
    
    assert p._option == true_option
    
def test_line():
    
    true_option = json.load(open('pandas-echarts/tests/data/line_option.json','r'))
    p = line(df,'year',y='lifeExp',category='continent',how='mean',stacked=True,area=True)
    
    assert p._option == true_option
    
def test_hist():
    
    true_option = json.load(open('pandas-echarts/tests/data/hist_option.json','r'))
    p = hist(df,'lifeExp',category='continent',bins=100,normed=True,stacked=True)
    
    assert p._option == true_option
    
def test_scatter():
    
    true_option = json.load(open('pandas-echarts/tests/data/scatter_option.json','r'))
    p = scatter(df[df.year == 1952],'lifeExp','gdpPercap',size='pop',category='continent')
    
    assert p._option == true_option

test_bar()
test_line()
test_hist()
test_scatter()

AssertionError: 

In [39]:
from bokeh import colors
from bokeh import palettes

In [23]:
true_option = json.load(open('pandas-echarts/tests/data/scatter_option.json','r'))

In [29]:
d = {}

In [30]:
d['label'] = {'name':'hellow'}

In [131]:
def foo(params):
    
    d.resync_data(df)
    return "Hello World"

In [132]:
d = bar(df,x='continent')
d.on_event('mouseover',foo)

In [48]:
ipywidgets.__version__

'4.1.1'

In [85]:
# %%file krisk/tests/test_make_chart.py
import pytest

@pytest.fixture
def chart_gapminder():
    import pandas as pd
    #from krisk import make_chart
    df = pd.read_csv('../echarts/gapminderDataFiveYear.txt',sep='\t')
    return (line(df,x='year',category='continent',y='lifeExp',how='mean',area=True,stacked=True)
             .set_theme('vintage')
             .set_tooltip())


def test_option(chart_gapminder):
    import json
    json_path ='krisk/tests/json_gapminder_line_year_continent_lifeExp_mean_area_stacked.json'
    option = json.load(open(json_path,'r'))
    assert option == chart_gapminder._option
    
test_option(chart_gapminder())

# TODO

Priority:

1. Legend and Title DONE
2. Tooltip DONE? Scatter

to_html DONE

set_toolbox

set_figsize()

refactor insert series DONE?

set item_style

self._data_columns = df.columns
if update:
    self._data.columns != df.columns

set annotate = True DONE

set ticks DONE?

set_color(background=None,palletes=None) DONE?

 set_title(title,pos=('center','top')/'left and set_legend

    'best'         : 0, (only implemented for axes legends)
    'upper right'  : 1,
    'upper left'   : 2,
    'lower left'   : 3,
    'lower right'  : 4,
    'right'        : 5,
    'center left'  : 6,
    'center right' : 7,
    'lower center' : 8,
    'upper center' : 9,
    'center'       : 10,

## bar
* http://echarts.baidu.com/demo.html#bar-y-category DONE
* http://echarts.baidu.com/demo.html#bar-y-category-stack DONE

## scatter
* http://echarts.baidu.com/demo.html#scatter-aqi-color
* http://echarts.baidu.com/demo.html#scatter-world-population
* http://echarts.baidu.com/demo.html#scatter-life-expectancy-timeline

## line
* http://echarts.baidu.com/demo.html#dynamic-data2 DONE
* http://echarts.baidu.com/demo.html#line-stack DONE
* http://echarts.baidu.com/demo.html#area-stack DONE

## Histogram hist with category

* http://echarts.baidu.com/demo.html#bar-animation-delay
* http://echarts.baidu.com/demo.html#mix-line-bar DONE
* http://echarts.baidu.com/demo.html#bar1(np.histogram, lose the markline and markpoint) DONE

### POTENTIALLY BUG

* callback for loading echarts

# Testing Echarts

In [193]:
%%javascript
$('#main').remove()
element.append('<div id="main" style="width: 600px;height:400px;"></div>');
require(['echarts'],function(echarts){
// 基于准备好的dom，初始化echarts实例
    
        var colorPalette = ['#d87c7c','#919e8b', '#d7ab82',  '#6e7074','#61a0a8','#efa18d', '#787464', '#cc7e63', '#724e58', '#4b565b'];
        echarts.registerTheme('vintage', {
            color: colorPalette,
            backgroundColor: '#fef8ef',
            graph: {
                color: colorPalette
            }
        });

        var myChart = echarts.init(document.getElementById('main'),'dark');

        // 指定图表的配置项和数据
        var option = {
            title: {
                text: 'ECharts 入门示例'
            },
            tooltip: {},
            legend: {
                data:['销量']
            },
            xAxis: {
                data: ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
            },
            yAxis: {},
            series: [{
                name: '销量',
                type: 'bar',
                data: [5, 20, 36, 10, 10, 20]
            }]
        };

        // 使用刚指定的配置项和数据显示图表。
        myChart.setOption(option);
});

"Hello World"

<IPython.core.display.Javascript object>

In [ ]:
# import json
# option = {
#         'title': {
#             'text': 'ECharts 入门示例'
#         },
#         'tooltip': {},
#         'legend': {
#             'data':['销量']
#         },
#         'xAxis': {
#             'data': ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
#         },
#         'yAxis': {},
#         'series': [{
#             'name': '销量',
#             'type': 'bar',
#             'data': [20, 20, 36, 10, 10, 20]
#         }]
#     }
# # Javascript("""var option={}""".format(json.dumps(option)))

In [23]:
from bokeh.io import output_notebook